In [1]:
import os
import json
import requests
import time
import re
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional, Tuple
from collections import Counter
from dotenv import load_dotenv

In [2]:
# Load environment variables
load_dotenv('keys.env')

True

In [3]:
# Configuration
class Config:
    # API Keys
    AMADEUS_API_KEY = os.getenv('AMADEUS_API_KEY')
    AMADEUS_API_SECRET = os.getenv('AMADEUS_API_SECRET')
    OPENWEATHER_API_KEY = os.getenv('OPENWEATHER_API_KEY')
    BOOKING_API_KEY = os.getenv('BOOKING_API_KEY')

    # Endpoints
    AMADEUS_AUTH_URL = "https://test.api.amadeus.com/v1/security/oauth2/token"
    AMADEUS_FLIGHT_SEARCH_URL = "https://test.api.amadeus.com/v2/shopping/flight-offers"
    AMADEUS_LOCATION_SEARCH_URL = "https://test.api.amadeus.com/v1/reference-data/locations"
    OPENWEATHER_URL = "https://api.openweathermap.org/data/2.5/forecast"
    TRAVEL_ADVISOR_RAPIDAPI_HOST = "booking-com15.p.rapidapi.com"

In [4]:
# Base Agent class
class Agent:
    def __init__(self, name: str):
        self.name = name
    
    def execute(self, *args, **kwargs):
        raise NotImplementedError("Agents must implement the execute method")
    
    def handle_error(self, error: Exception) -> Dict:
        return {"status": "error", "message": str(error)}

In [5]:
# Flight API Agent
class FlightAPIAgent(Agent):
    def __init__(self):
        super().__init__("Flight")
        self.access_token = None
        self.token_expiry = None
        self.city_code_cache = {}
    
    def _get_access_token(self) -> str:
        if self.access_token and self.token_expiry and datetime.now() < self.token_expiry:
            return self.access_token
        
        try:
            response = requests.post(
                Config.AMADEUS_AUTH_URL,
                data={
                    'grant_type': 'client_credentials',
                    'client_id': Config.AMADEUS_API_KEY,
                    'client_secret': Config.AMADEUS_API_SECRET
                },
                headers={'Content-Type': 'application/x-www-form-urlencoded'}
            )
            
            if response.status_code != 200:
                raise Exception(f"Failed to authenticate: {response.status_code} - {response.text}")
            
            data = response.json()
            self.access_token = data['access_token']
            self.token_expiry = datetime.now() + timedelta(seconds=data['expires_in'] - 60)
            return self.access_token
        except Exception as e:
            raise Exception(f"Amadeus authentication failed: {str(e)}")
    
    def _get_city_code(self, city: str) -> str:
        if not city:
            return None
            
        city_parts = city.lower().split()
        if len(city_parts) > 1:
            if city_parts[-1] in ["from", "on", "to", "in", "for", "with"]:
                city_parts = city_parts[:-1]
        
        city_clean = " ".join(city_parts)
        
        if city_clean in self.city_code_cache:
            return self.city_code_cache[city_clean]
        
        if len(city_clean) == 3 and city_clean.upper() == city_clean:
            return city_clean
        
        try:
            token = self._get_access_token()
            max_retries = 2
            for attempt in range(max_retries):
                try:
                    url = Config.AMADEUS_LOCATION_SEARCH_URL
                    headers = {"Authorization": f"Bearer {token}"}
                    params = {"keyword": city_clean, "subType": "CITY,AIRPORT"}
                    response = requests.get(url, headers=headers, params=params, timeout=10)
                    
                    if response.status_code == 200:
                        data = response.json()
                        if data.get('data') and len(data['data']) > 0:
                            iata_code = data['data'][0]['iataCode']
                            self.city_code_cache[city_clean] = iata_code
                            return iata_code
                    
                    elif attempt == 0 and ' ' in city_clean:
                        city_clean = city_clean.split()[0]
                        continue
                    
                    break
                
                except requests.exceptions.RequestException:
                    if attempt == max_retries - 1:
                        break
                    time.sleep(1)
        
        except Exception:
            pass
        
        city_code = self._derive_airport_code(city_clean)
        self.city_code_cache[city_clean] = city_code
        return city_code
    
    def _derive_airport_code(self, city: str) -> str:
        clean_city = re.sub(r'[^a-zA-Z]', '', city)
        if len(clean_city) < 3:
            padding = "X" * (3 - len(clean_city))
            city_code = (clean_city + padding).upper()
        else:
            city_code = clean_city[:3].upper()
        return city_code
    
    def _validate_dates(self, departure_date: str, return_date: Optional[str] = None) -> Tuple[str, Optional[str]]:
        try:
            today = datetime.now().date()
            departure = datetime.strptime(departure_date, "%Y-%m-%d").date()
            
            if departure < today:
                departure = today + timedelta(days=1)
            
            if return_date:
                return_date_obj = datetime.strptime(return_date, "%Y-%m-%d").date()
                if return_date_obj <= departure:
                    return_date_obj = departure + timedelta(days=1)
                return departure.strftime("%Y-%m-%d"), return_date_obj.strftime("%Y-%m-%d")
            
            return departure.strftime("%Y-%m-%d"), return_date
            
        except ValueError as e:
            raise ValueError(f"Invalid date format: {str(e)}. Please use YYYY-MM-DD format.")
    
    def execute(self, origin: str, destination: str, departure_date: str, 
                return_date: Optional[str] = None, adults: int = 1) -> Dict:
        try:
            token = self._get_access_token()
            
            try:
                departure_date, return_date = self._validate_dates(departure_date, return_date)
            except ValueError as date_error:
                return {"status": "error", "message": str(date_error)}
            
            origin_code = self._get_city_code(origin)
            destination_code = self._get_city_code(destination)
            
            if not origin_code or not destination_code:
                return {"status": "error", "message": f"Could not determine IATA codes for {origin} or {destination}"}
            
            params = {
                'originLocationCode': origin_code,
                'destinationLocationCode': destination_code,
                'departureDate': departure_date,
                'adults': adults,
                'max': 5,
                'currencyCode': 'USD'
            }
            
            if return_date:
                params['returnDate'] = return_date
            
            headers = {'Authorization': f'Bearer {token}'}
            
            response = requests.get(
                Config.AMADEUS_FLIGHT_SEARCH_URL,
                params=params,
                headers=headers,
                timeout=15
            )

            if response.status_code != 200:
                error_message = f"Flight search failed: {response.status_code}"
                try:
                    error_data = response.json()
                    if 'errors' in error_data and error_data['errors']:
                        error_details = []
                        for error in error_data['errors']:
                            if 'detail' in error:
                                error_details.append(error['detail'])
                            elif 'title' in error:
                                error_details.append(error['title'])
                        if error_details:
                            error_message = f"Flight search failed: {', '.join(error_details)}"
                except Exception:
                    error_message = f"Flight search failed: {response.status_code} - {response.text}"
                return {"status": "error", "message": error_message}
            
            data = response.json()
            
            if 'data' not in data or not data['data']:
                return {"status": "success", "message": "No flights found for the specified criteria", "flights": []}
            
            flights = []
            for offer in data.get('data', []):
                price_info = offer.get('price', {})
                price = f"{price_info.get('total', 'N/A')} {price_info.get('currency', 'USD')}"
                
                flight_info = {'price': price, 'segments': []}
                
                for itinerary in offer.get('itineraries', []):
                    for segment in itinerary.get('segments', []):
                        duration = segment.get('duration', '')
                        if duration.startswith('PT'):
                            duration = duration[2:].replace('H', 'h ').replace('M', 'm').strip()
                        
                        segment_info = {
                            'airline': segment.get('carrierCode', 'Unknown'),
                            'flight_number': segment.get('number', 'Unknown'),
                            'departure': {
                                'airport': segment.get('departure', {}).get('iataCode', 'Unknown'),
                                'time': segment.get('departure', {}).get('at', 'Unknown')
                            },
                            'arrival': {
                                'airport': segment.get('arrival', {}).get('iataCode', 'Unknown'),
                                'time': segment.get('arrival', {}).get('at', 'Unknown')
                            },
                            'duration': duration
                        }
                        flight_info['segments'].append(segment_info)
                
                flights.append(flight_info)
            
            return {"status": "success", "flights": flights}
            
        except requests.exceptions.Timeout:
            return {"status": "error", "message": "Flight search timed out. Please try again later."}
        except requests.exceptions.ConnectionError:
            return {"status": "error", "message": "Connection error while searching for flights. Please check your internet connection."}
        except Exception as e:
            return self.handle_error(e)

In [6]:
# Weather API Agent
class WeatherAPIAgent(Agent):
    def __init__(self, api_key: str):
        super().__init__("Weather")
        self.api_key = api_key
        self.base_url = "https://api.openweathermap.org/data/2.5/forecast"

    def clean_city_name(self, city: str) -> str:
        if not city:
            return city
        city_clean = city.lower().strip()
        prepositions = ["from", "on", "to", "in", "for", "with"]
        for prep in prepositions:
            if city_clean.endswith(f" {prep}"):
                city_clean = city_clean[:-len(prep) - 1]
        city_clean = city_clean.strip(",.;:")
        return city_clean

    def execute(self, city: str, start_date: str, end_date: str) -> Dict:
        try:
            city_clean = self.clean_city_name(city)
            if not city_clean:
                return {"status": "error", "message": "Invalid or empty city name provided"}

            params = {
                "q": city_clean,
                "appid": self.api_key,
                "units": "metric",
                "cnt": 40
            }

            response = requests.get(self.base_url, params=params)
            
            if response.status_code == 404:
                return {"status": "error", "message": f"City '{city_clean}' not found"}
            elif response.status_code == 401:
                return {"status": "error", "message": "Invalid API key"}
            elif response.status_code != 200:
                return {"status": "error", "message": f"Weather API request failed: {response.status_code} - {response.text}"}

            data = response.json()

            if "list" not in data or not data["list"]:
                return {"status": "error", "message": "No forecast data available in API response"}

            try:
                start = datetime.strptime(start_date, "%Y-%m-%d")
                end = datetime.strptime(end_date or start_date, "%Y-%m-%d")
            except ValueError:
                return {"status": "error", "message": "Invalid date format. Use YYYY-MM-DD"}

            daily_forecasts = {}
            for forecast in data.get("list", []):
                forecast_time = datetime.fromtimestamp(forecast["dt"])
                forecast_date = forecast_time.date()

                if start.date() <= forecast_date <= end.date():
                    if forecast_date not in daily_forecasts:
                        daily_forecasts[forecast_date] = {
                            "temps": [],
                            "conditions": [],
                            "humidity": []
                        }

                    daily_data = daily_forecasts[forecast_date]
                    daily_data["temps"].append(forecast["main"]["temp"])
                    daily_data["conditions"].append(forecast["weather"][0]["main"])
                    daily_data["humidity"].append(forecast["main"]["humidity"])

            weather_forecast = []
            for date, data in sorted(daily_forecasts.items()):
                temps = data["temps"]
                conditions = data["conditions"]
                humidity = data["humidity"]

                daily_summary = {
                    "date": date.strftime("%Y-%m-%d"),
                    "min_temp": min(temps) if temps else None,
                    "max_temp": max(temps) if temps else None,
                    "avg_temp": sum(temps) / len(temps) if temps else None,
                    "condition": Counter(conditions).most_common(1)[0][0] if conditions else "Unknown",
                    "avg_humidity": sum(humidity) / len(humidity) if humidity else None
                }
                weather_forecast.append(daily_summary)

            if not weather_forecast:
                return {
                    "status": "success",
                    "message": "No weather forecast available for the specified dates",
                    "city": city_clean,
                    "forecast": []
                }

            return {
                "status": "success",
                "city": data.get("city", {}).get("name", city_clean),
                "forecast": weather_forecast
            }

        except Exception as e:
            return self.handle_error(e)

In [7]:
# Hotel API Agent
class HotelAPIAgent(Agent):
    def __init__(self):
        super().__init__("Hotel")
        self.base_url = "https://booking-com15.p.rapidapi.com/api/v1/hotels/searchHotelsByCoordinates"
        self.geocoding_url = "http://api.openweathermap.org/geo/1.0/direct"
        self.reverse_geocoding_url = "https://nominatim.openstreetmap.org/reverse"
        self.api_key = Config.BOOKING_API_KEY
        self.api_host = Config.TRAVEL_ADVISOR_RAPIDAPI_HOST
        self.weather_api_key = Config.OPENWEATHER_API_KEY

    def get_coordinates(self, city: str) -> tuple:
        params = {
            "q": city,
            "limit": 1,
            "appid": self.weather_api_key
        }
        try:
            response = requests.get(self.geocoding_url, params=params)
            response.raise_for_status()
            data = response.json()
            if data:
                return data[0]["lat"], data[0]["lon"]
            return None, None
        except requests.exceptions.RequestException:
            return None, None

    def get_address_from_coordinates(self, latitude: float, longitude: float) -> str:
        try:
            params = {
                "lat": latitude,
                "lon": longitude,
                "format": "json",
                "zoom": 18
            }
            headers = {
                "User-Agent": "TravelAssistant/1.0 (contact: your_email@example.com)"
            }
            response = requests.get(self.reverse_geocoding_url, params=params, headers=headers, timeout=5)
            response.raise_for_status()
            data = response.json()
            return data.get("display_name", "Unknown address")
        except requests.exceptions.RequestException:
            return "Unknown address"

    def execute(self, city: str, check_in: str, check_out: str, adults: int = 1) -> Dict:
        try:
            latitude, longitude = self.get_coordinates(city)
            if latitude is None or longitude is None:
                return {"status": "error", "message": f"Could not find coordinates for {city}"}

            headers = {
                "X-RapidAPI-Key": self.api_key,
                "X-RapidAPI-Host": self.api_host
            }
            params = {
                "latitude": latitude,
                "longitude": longitude,
                "arrival_date": check_in,
                "departure_date": check_out,
                "adults": adults,
                "children": 0,
                "room_qty": 1,
                "units": "metric",
                "page_number": 1,
                "languagecode": "en-us",
                "currency_code": "EUR",
                "location": "US",
                "limit": 10
            }
            response = requests.get(self.base_url, headers=headers, params=params)
            response.raise_for_status()
            data = response.json()
            hotels_data = data.get("data", {}).get("result", [])

            if not isinstance(hotels_data, list):
                return {"status": "error", "message": f"Invalid response structure: {hotels_data}"}

            hotels = []
            for hotel in hotels_data[:3]:
                gross_price = hotel.get("min_total_price", "N/A")
                currency = hotel.get("currencycode", "EUR")
                latitude = hotel.get("latitude")
                longitude = hotel.get("longitude")
                
                address = self.get_address_from_coordinates(latitude, longitude) if latitude and longitude else "Unknown address"
                
                hotels.append({
                    "name": hotel.get("hotel_name", "Unknown Hotel"),
                    "price": f"{gross_price} {currency}" if gross_price != "N/A" else "N/A",
                    "rating": hotel.get("review_score", "N/A"),
                    "address": address
                })
                time.sleep(1)

            return {"status": "success", "hotels": hotels}
        except requests.exceptions.RequestException as e:
            return self.handle_error(e)


In [8]:
# Itinerary Planner Agent
class ItineraryPlannerAgent(Agent):
    def __init__(self):
        super().__init__("Itinerary Planner")
        self.flight_agent = FlightAPIAgent()
        self.weather_agent = WeatherAPIAgent(Config.OPENWEATHER_API_KEY)
        self.hotel_agent = HotelAPIAgent()
    
    def _normalize_dates(self, departure_date: str, return_date: Optional[str] = None) -> Tuple[str, Optional[str]]:
        try:
            today = datetime.now().date()
            
            try:
                departure = datetime.strptime(departure_date, "%Y-%m-%d").date()
            except ValueError:
                try:
                    departure = datetime.strptime(departure_date, "%m/%d/%Y").date()
                except ValueError:
                    departure = datetime.strptime(departure_date, "%d-%m-%Y").date()
            
            if departure < today:
                departure = today + timedelta(days=1)
            
            if return_date:
                try:
                    ret_date = datetime.strptime(return_date, "%Y-%m-%d").date()
                except ValueError:
                    try:
                        ret_date = datetime.strptime(return_date, "%m/%d/%Y").date()
                    except ValueError:
                        ret_date = datetime.strptime(return_date, "%d-%m-%Y").date()
                
                if ret_date <= departure:
                    ret_date = departure + timedelta(days=1)
                
                return departure.strftime("%Y-%m-%d"), ret_date.strftime("%Y-%m-%d")
            
            return departure.strftime("%Y-%m-%d"), return_date
        except Exception as e:
            tomorrow = (today + timedelta(days=1)).strftime("%Y-%m-%d")
            day_after = (today + timedelta(days=2)).strftime("%Y-%m-%d")
            return tomorrow, day_after if return_date else None
    
    def _clean_city_name(self, city: str) -> str:
        if not city:
            return ""
        
        city_parts = city.split()
        if len(city_parts) > 1 and city_parts[-1].lower() in ["from", "to", "on", "in", "for", "with"]:
            city_parts = city_parts[:-1]
        
        clean_city = " ".join(city_parts)
        clean_city = re.sub(r'[,.;:!?]$', '', clean_city).strip()
        
        return clean_city
    
    def execute(self, query: Dict) -> Dict:
        try:
            origin = self._clean_city_name(query.get('origin', ''))
            destination = self._clean_city_name(query.get('destination', ''))
            departure_date = query.get('departure_date')
            return_date = query.get('return_date')
            adults = query.get('adults', 1)
            
            if not origin or not destination or not departure_date:
                return {"status": "error", "message": "Missing required parameters: origin, destination, and departure_date"}
            
            try:
                normalized_departure, normalized_return = self._normalize_dates(departure_date, return_date)
            except Exception as e:
                return {"status": "error", "message": f"Invalid date format: {str(e)}"}
            
            flights_result = self.flight_agent.execute(
                origin=origin,
                destination=destination,
                departure_date=normalized_departure,
                return_date=normalized_return,
                adults=adults
            )
            
            weather_result = self.weather_agent.execute(
                city=destination,
                start_date=normalized_departure,
                end_date=normalized_return or normalized_departure
            )
            
            hotels_result = self.hotel_agent.execute(
                city=destination,
                check_in=normalized_departure,
                check_out=normalized_return or (
                    datetime.strptime(normalized_departure, "%Y-%m-%d") + timedelta(days=1)
                ).strftime("%Y-%m-%d"),
                adults=adults
            )
            
            itinerary = {
                "status": "success",
                "travel_details": {
                    "origin": origin,
                    "destination": destination,
                    "departure_date": normalized_departure,
                    "return_date": normalized_return,
                    "adults": adults
                },
                "flights": [],
                "weather": [],
                "hotels": []
            }
            
            if flights_result.get("status") == "success":
                itinerary["flights"] = flights_result.get("flights", [])
            else:
                itinerary["flight_error"] = flights_result.get("message", "Unknown flight error")
                if "INVALID DATE" in str(flights_result.get("message", "")):
                    itinerary["flight_error"] += " Please select a future travel date."
            
            if weather_result.get("status") == "success":
                itinerary["weather"] = weather_result.get("forecast", [])
            else:
                itinerary["weather_error"] = weather_result.get("message", "Unknown weather error")
            
            if hotels_result.get("status") == "success":
                itinerary["hotels"] = hotels_result.get("hotels", [])
            else:
                itinerary["hotel_error"] = hotels_result.get("message", "Unknown hotel error")
            
            return itinerary
        except Exception as e:
            return self.handle_error(e)

In [9]:
# Travel Query Processor
class TravelQueryProcessor:
    def __init__(self):
        self.iso_date_pattern = re.compile(r'\d{4}-\d{2}-\d{2}')
        self.alt_date_patterns = [
            re.compile(r'\b(0?[1-9]|1[0-2])/(0?[1-9]|[12]\d|3[01])/(\d{4})\b'),
            re.compile(r'\b(0?[1-9]|[12]\d|3[01])/(0?[1-9]|1[0-2])/(\d{4})\b'),
            re.compile(r'\b(0?[1-9]|[12]\d|3[01])-(0?[1-9]|1[0-2])-(\d{4})\b'),
            re.compile(r'\b(0?[1-9]|1[0-2])-(0?[1-9]|[12]\d|3[01])-(\d{4})\b')
        ]
        self.adults_pattern = re.compile(r'(\d+)\s*(?:adult|adults|people|persons|travelers|travellers)')
        self.city_prefixes = [
            'from', 'to', 'in', 'visit', 'visiting', 'going to', 'departing from',
            'leaving from', 'arriving in', 'arriving at', 'heading to'
        ]
    
    def _standardize_date(self, date_str: str) -> str:
        if re.match(self.iso_date_pattern, date_str):
            return date_str
        
        for i, pattern in enumerate(self.alt_date_patterns):
            match = pattern.match(date_str)
            if match:
                if i in [0, 3]:
                    month, day, year = match.groups()
                else:
                    day, month, year = match.groups()
                
                day = day.zfill(2)
                month = month.zfill(2)
                
                return f"{year}-{month}-{day}"
        
        return date_str
    
    def _extract_dates(self, text: str) -> List[str]:
        dates = []
        
        iso_dates = self.iso_date_pattern.findall(text)
        dates.extend(iso_dates)
        
        for i, pattern in enumerate(self.alt_date_patterns):
            matches = pattern.findall(text)
            for match in matches:
                if i in [0, 3]:
                    month, day, year = match
                else:
                    day, month, year = match
                
                day = day.zfill(2)
                month = month.zfill(2)
                
                standardized = f"{year}-{month}-{day}"
                if standardized not in dates:
                    dates.append(standardized)
        
        return dates
    
    def _find_locations(self, text: str) -> Tuple[Optional[str], Optional[str]]:
        text_lower = text.lower()
        origin = None
        destination = None
        
        from_to_matches = re.findall(r'from\s+([a-zA-Z\s]+)\s+to\s+([a-zA-Z\s]+)', text_lower)
        if from_to_matches:
            origin_text, dest_text = from_to_matches[0]
            origin = self._clean_location_name(origin_text)
            destination = self._clean_location_name(dest_text)
            return origin, destination
        
        for prefix in self.city_prefixes:
            pattern = f"{prefix}\\s+([a-zA-Z\\s]+?)(?:\\s+to|\\s+from|\\s+on|\\s+in|\\s+for|\\s+with|\\s+\\d|\\s*$)"
            matches = re.findall(pattern, text_lower)
            
            if matches:
                location = self._clean_location_name(matches[0])
                if prefix in ['from', 'departing from', 'leaving from']:
                    origin = location
                else:
                    destination = location
        
        if not origin or not destination:
            city_pattern = r'(?:^|\W)([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)(?:\W|$)'
            city_candidates = re.findall(city_pattern, text)
            
            if len(city_candidates) >= 2:
                if not origin:
                    origin = city_candidates[0]
                if not destination:
                    destination = city_candidates[1]
        
        return origin, destination
    
    def _clean_location_name(self, location: str) -> str:
        if not location:
            return ""
        
        location_parts = location.split()
        if location_parts and location_parts[-1] in ["from", "to", "on", "in", "for", "with"]:
            location_parts = location_parts[:-1]
        
        location = " ".join(location_parts)
        location = re.sub(r'[,.;:!?]$', '', location).strip()
        return location
    
    def extract_travel_details(self, query_text: str) -> Optional[Dict]:
        query_text = query_text.strip()
        if not query_text:
            return None
        
        dates = self._extract_dates(query_text)
        departure_date = dates[0] if dates else None
        return_date = dates[1] if len(dates) > 1 else None
        
        origin, destination = self._find_locations(query_text)
        
        adults = 1
        adults_match = re.search(self.adults_pattern, query_text.lower())
        if adults_match:
            adults = int(adults_match.group(1))
        
        if not origin or not destination or not departure_date:
            return None
        
        return {
            "origin": origin,
            "destination": destination,
            "departure_date": departure_date,
            "return_date": return_date,
            "adults": adults
        }

In [10]:
# Travel Assistant (main class)
class TravelAssistant:
    def __init__(self):
        self.query_processor = TravelQueryProcessor()
        self.itinerary_agent = ItineraryPlannerAgent()
    
    def process_query(self, query_text: str) -> Dict:
        travel_details = self.query_processor.extract_travel_details(query_text)
        
        if not travel_details:
            return {
                "status": "error",
                "message": "Could not understand the travel query. Please provide details like origin, destination, and dates."
            }
        
        return self.itinerary_agent.execute(travel_details)

In [11]:
# Itinerary Formatter
class ItineraryFormatter:
    @staticmethod
    def format_itinerary(itinerary: Dict) -> str:
        if itinerary.get("status") != "success":
            return f"Error: {itinerary.get('message', 'Unknown error')}"
        
        details = itinerary.get("travel_details", {})
        origin = details.get("origin", "Unknown")
        destination = details.get("destination", "Unknown")
        departure_date = details.get("departure_date", "Unknown")
        return_date = details.get("return_date", "Unknown")
        adults = details.get("adults", 1)
        
        formatted = []
        formatted.append(f"TRAVEL ITINERARY: {origin.title()} to {destination.title()}")
        formatted.append("=" * 50)
        formatted.append(f"Travelers: {adults} {'adult' if adults == 1 else 'adults'}")
        formatted.append(f"Departure: {departure_date}")
        if return_date:
            formatted.append(f"Return: {return_date}")
        formatted.append("")
        
        formatted.append("FLIGHT OPTIONS")
        formatted.append("-" * 50)
        
        flights = itinerary.get("flights", [])
        if flights:
            for i, flight in enumerate(flights[:3], 1):
                formatted.append(f"Option {i}: {flight.get('price', 'Price not available')}")
                
                for j, segment in enumerate(flight.get("segments", []), 1):
                    departure = segment.get("departure", {})
                    arrival = segment.get("arrival", {})
                    
                    formatted.append(f"  Segment {j}: {segment.get('airline', '')} {segment.get('flight_number', '')}")
                    formatted.append(f"    {departure.get('airport', '')} → {arrival.get('airport', '')}")
                    formatted.append(f"    {departure.get('time', '')} → {arrival.get('time', '')}")
                    formatted.append(f"    Duration: {segment.get('duration', '')}")
                
                formatted.append("")
        else:
            if "flight_error" in itinerary:
                formatted.append(f"Flight information unavailable: {itinerary.get('flight_error')}")
            else:
                formatted.append("No flight options found for your criteria.")
        formatted.append("")
        
        formatted.append("WEATHER FORECAST")
        formatted.append("-" * 50)
        
        weather = itinerary.get("weather", [])
        if weather:
            for forecast in weather:
                date = forecast.get("date", "Unknown date")
                min_temp = forecast.get("min_temp")
                max_temp = forecast.get("max_temp")
                condition = forecast.get("condition", "Unknown")
                
                temp_range = ""
                if min_temp is not None and max_temp is not None:
                    temp_range = f"{min_temp:.1f}°C to {max_temp:.1f}°C"
                elif max_temp is not None:
                    temp_range = f"Up to {max_temp:.1f}°C"
                elif min_temp is not None:
                    temp_range = f"At least {min_temp:.1f}°C"
                
                formatted.append(f"{date}: {condition}, {temp_range}")
        else:
            if "weather_error" in itinerary:
                formatted.append(f"Weather information unavailable: {itinerary.get('weather_error')}")
            else:
                formatted.append("No weather forecast available for your destination.")
        formatted.append("")
        
        formatted.append("HOTEL OPTIONS")
        formatted.append("-" * 50)
        
        hotels = itinerary.get("hotels", [])
        if hotels:
            for i, hotel in enumerate(hotels, 1):
                formatted.append(f"Option {i}: {hotel.get('name', 'Unknown hotel')}")
                formatted.append(f"  Rating: {hotel.get('rating', 'N/A')}")
                formatted.append(f"  Price: {hotel.get('price', 'N/A')}")
                formatted.append(f"  Address: {hotel.get('address', 'N/A')}")
                
                amenities = hotel.get("amenities", [])
                if amenities:
                    formatted.append(f"  Amenities: {', '.join(amenities)}")
                
                formatted.append("")
        else:
            if "hotel_error" in itinerary:
                formatted.append(f"Hotel information unavailable: {itinerary.get('hotel_error')}")
            else:
                formatted.append("No hotel options found for your destination.")
        
        return "\n".join(formatted)

In [12]:
# Example usage function
def run_example():
    assistant = TravelAssistant()
    formatter = ItineraryFormatter()
    
    today = datetime.now()
    future_date1 = (today + timedelta(days=1)).strftime("%Y-%m-%d")
    future_date2 = (today + timedelta(days=4)).strftime("%Y-%m-%d")
    
    query = f"I want to travel from San Francisco to Hyderabad from {future_date1} to {future_date2} with 2 adults"
    
    result = assistant.process_query(query)
    formatted_itinerary = formatter.format_itinerary(result)
    print(formatted_itinerary)

# Main execution
if __name__ == "__main__":
    run_example()

TRAVEL ITINERARY: San Francisco to Hyderabad
Travelers: 2 adults
Departure: 2025-05-03
Return: 2025-05-06

FLIGHT OPTIONS
--------------------------------------------------
Option 1: 1936.54 USD
  Segment 1: AI 184
    SFO → DEL
    2025-05-03T22:00:00 → 2025-05-05T02:30:00
    Duration: 16h
  Segment 2: AI 2829
    DEL → HYD
    2025-05-05T06:10:00 → 2025-05-05T08:25:00
    Duration: 2h 15m
  Segment 3: AI 9569
    HYD → BLR
    2025-05-06T23:00:00 → 2025-05-07T00:15:00
    Duration: 1h 15m
  Segment 4: AI 175
    BLR → SFO
    2025-05-07T12:50:00 → 2025-05-07T17:30:00
    Duration: 17h 10m

Option 2: 1936.54 USD
  Segment 1: AI 184
    SFO → DEL
    2025-05-03T22:00:00 → 2025-05-05T02:30:00
    Duration: 16h
  Segment 2: AI 2560
    DEL → HYD
    2025-05-05T07:10:00 → 2025-05-05T09:30:00
    Duration: 2h 20m
  Segment 3: AI 9569
    HYD → BLR
    2025-05-06T23:00:00 → 2025-05-07T00:15:00
    Duration: 1h 15m
  Segment 4: AI 175
    BLR → SFO
    2025-05-07T12:50:00 → 2025-05-07T17:30

In [13]:
# Example usage function
def run_example():
    assistant = TravelAssistant()
    formatter = ItineraryFormatter()
    
    today = datetime.now()
    future_date1 = (today + timedelta(days=15)).strftime("%Y-%m-%d")
    future_date2 = (today + timedelta(days=20)).strftime("%Y-%m-%d")
    
    query = f"I want to travel from New York to London from {future_date1} to {future_date2} with 2 adults"
    
    result = assistant.process_query(query)
    formatted_itinerary = formatter.format_itinerary(result)
    print(formatted_itinerary)

# Main execution
if __name__ == "__main__":
    run_example()

TRAVEL ITINERARY: New York to London
Travelers: 2 adults
Departure: 2025-05-17
Return: 2025-05-22

FLIGHT OPTIONS
--------------------------------------------------
Option 1: 1069.82 USD
  Segment 1: FI 618
    JFK → KEF
    2025-05-17T23:10:00 → 2025-05-18T08:55:00
    Duration: 5h 45m
  Segment 2: FI 472
    KEF → LGW
    2025-05-18T10:30:00 → 2025-05-18T14:40:00
    Duration: 3h 10m
  Segment 3: FI 473
    LGW → KEF
    2025-05-22T15:40:00 → 2025-05-22T17:55:00
    Duration: 3h 15m
  Segment 4: FI 619
    KEF → JFK
    2025-05-22T19:55:00 → 2025-05-22T22:10:00
    Duration: 6h 15m

Option 2: 1137.62 USD
  Segment 1: FI 618
    JFK → KEF
    2025-05-17T23:10:00 → 2025-05-18T08:55:00
    Duration: 5h 45m
  Segment 2: FI 472
    KEF → LGW
    2025-05-18T10:30:00 → 2025-05-18T14:40:00
    Duration: 3h 10m
  Segment 3: FI 451
    LHR → KEF
    2025-05-22T13:05:00 → 2025-05-22T15:15:00
    Duration: 3h 10m
  Segment 4: FI 623
    KEF → EWR
    2025-05-22T17:00:00 → 2025-05-22T19:10:00
   

In [14]:
# Example usage function
def run_example():
    assistant = TravelAssistant()
    formatter = ItineraryFormatter()
    
    today = datetime.now()
    future_date1 = (today + timedelta(days=3)).strftime("%Y-%m-%d")
    future_date2 = (today + timedelta(days=5)).strftime("%Y-%m-%d")
    
    query = f"I want to travel from Moscow to Jakarta from {future_date1} to {future_date2} with 2 adults"
    
    result = assistant.process_query(query)
    formatted_itinerary = formatter.format_itinerary(result)
    print(formatted_itinerary)

# Main execution
if __name__ == "__main__":
    run_example()

TRAVEL ITINERARY: Moscow to Jakarta
Travelers: 2 adults
Departure: 2025-05-05
Return: 2025-05-07

FLIGHT OPTIONS
--------------------------------------------------
No flight options found for your criteria.

WEATHER FORECAST
--------------------------------------------------
2025-05-05: Clouds, 27.3°C to 33.1°C
2025-05-06: Rain, 27.0°C to 32.3°C

HOTEL OPTIONS
--------------------------------------------------
Option 1: Model J Hotel Jakarta Soekarno - Hatta Airport 曼居雅加达苏加诺哈达机场酒店
  Rating: 9
  Price: 1300000 IDR
  Address: RW 05, Cengkareng Business City, Benda, Tangerang, Banten, Jawa, 15125, Indonesia

Option 2: BRANZ BSD Prime Location - BSD Central Business District & ICE BSD
  Rating: 9
  Price: 1438832 IDR
  Address: Branz Marketing Office, Terowongan CBD BSD, BSD Grand CBD, BSD City, Pagedangan, Kabupaten Tangerang, Banten, Jawa, 15345, Indonesia

Option 3: U Residence Tower2 Lippo Karawaci by supermal
  Rating: 8.1
  Price: 1109700 IDR
  Address: Supermall Karawaci, 105, Jalan

In [15]:
# Example usage function
def run_example():
    assistant = TravelAssistant()
    formatter = ItineraryFormatter()
    
    today = datetime.now()
    future_date1 = (today + timedelta(days=10)).strftime("%Y-%m-%d")
    future_date2 = (today + timedelta(days=15)).strftime("%Y-%m-%d")
    
    query = f"I want to travel from Cape Town to Beijing from {future_date1} to {future_date2} with 2 adults"
    
    result = assistant.process_query(query)
    formatted_itinerary = formatter.format_itinerary(result)
    print(formatted_itinerary)

# Main execution
if __name__ == "__main__":
    run_example()

TRAVEL ITINERARY: Cape Town to Beijing
Travelers: 2 adults
Departure: 2025-05-12
Return: 2025-05-17

FLIGHT OPTIONS
--------------------------------------------------
Option 1: 12654.64 USD
  Segment 1: S6 608
    CAP → MIA
    2025-05-12T11:00:00 → 2025-05-12T12:50:00
    Duration: 1h 50m
  Segment 2: AS 305
    MIA → SEA
    2025-05-12T18:01:00 → 2025-05-12T21:59:00
    Duration: 6h 58m
  Segment 3: HU 496
    SEA → PEK
    2025-05-13T11:00:00 → 2025-05-14T15:10:00
    Duration: 13h 10m
  Segment 4: HU 495
    PEK → SEA
    2025-05-17T13:10:00 → 2025-05-17T09:00:00
    Duration: 10h 50m
  Segment 5: AS 305
    SEA → MIA
    2025-05-18T08:25:00 → 2025-05-18T17:21:00
    Duration: 5h 56m
  Segment 6: S6 609
    MIA → CAP
    2025-05-19T08:00:00 → 2025-05-19T09:50:00
    Duration: 1h 50m


WEATHER FORECAST
--------------------------------------------------
No weather forecast available for your destination.

HOTEL OPTIONS
--------------------------------------------------
Option 1: Muti